In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
covid_data = pd.read_csv('COVID-QA_Dataset.csv')
covid_data.head()

,Question,Answer,Source,Date Accessed
0,When was the Middle East Respiratory Syndrome ...,"(MERS-CoV) was first isolated in 2012, in a 60...",GitHub COVID-QA Dataset,27-Apr-20
1,Which medical comorbidities most profoundly in...,"if they were obese, immunocompromised, diabeti...",GitHub COVID-QA Dataset,27-Apr-20
2,What is the size of bovine coronavirus,31 kb,GitHub COVID-QA Dataset,27-Apr-20
3,What is the estimated R0 of COVID-19,2.2,GitHub COVID-QA Dataset,27-Apr-20
4,What is the relationship between SARS-CoV and ...,Many infected patients have acute lung injury ...,GitHub COVID-QA Dataset,27-Apr-20


In [3]:
covid_data = covid_data.drop(['Source','Date Accessed'], axis = 1)
covid_data

,Question,Answer
0,When was the Middle East Respiratory Syndrome ...,"(MERS-CoV) was first isolated in 2012, in a 60..."
1,Which medical comorbidities most profoundly in...,"if they were obese, immunocompromised, diabeti..."
2,What is the size of bovine coronavirus,31 kb
3,What is the estimated R0 of COVID-19,2.2
4,What is the relationship between SARS-CoV and ...,Many infected patients have acute lung injury ...
...,...,...
190,What suggests that Irish equine coronaviruses ...,high level of homology between viruses
191,Where was SARS-CoV-2 first identified,"Wuhan, capital of Hubei Province, China"
192,What is a major difference in clinical progres...,"Unlike SARS-CoV infection, which causes primar..."
193,How big was the temporary hospital built in Wu...,1000 bed hospital


In [4]:
covid_data.head()

,Question,Answer
0,When was the Middle East Respiratory Syndrome ...,"(MERS-CoV) was first isolated in 2012, in a 60..."
1,Which medical comorbidities most profoundly in...,"if they were obese, immunocompromised, diabeti..."
2,What is the size of bovine coronavirus,31 kb
3,What is the estimated R0 of COVID-19,2.2
4,What is the relationship between SARS-CoV and ...,Many infected patients have acute lung injury ...


In [14]:
def formatting(ques, ans, num):
    return f"{num}. Question: {ques}\n    Answer: {ans}"

In [15]:
formatted_data = []

In [16]:
for index, row in covid_data.iterrows():
    ques = row[0]
    ans = row[1]
    formatted_entry = formatting(ques, ans, index + 1)
    formatted_data.append([formatted_entry])

/var/folders/b5/1td5nx5s3x53rhk3l8jcrkwr0000gn/T/ipykernel_30249/1706605143.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ques = row[0]
/var/folders/b5/1td5nx5s3x53rhk3l8jcrkwr0000gn/T/ipykernel_30249/1706605143.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ans = row[1]


In [17]:
import csv

In [18]:
with open('formatted_output.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Q&A"]) 
    writer.writerows(formatted_data)

In [19]:
llm_data = pd.read_csv('formatted_output.csv')
llm_data.head()

,Q&A
0,1. Question: When was the Middle East Respirat...
1,2. Question: Which medical comorbidities most ...
2,3. Question: What is the size of bovine corona...
3,4. Question: What is the estimated R0 of COVID...
4,5. Question: What is the relationship between ...


In [38]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

tags_covid_data = pd.read_csv('formatted_output.csv')
texts = tags_covid_data['Q&A'].astype(str)
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(texts)
num_clusters = 5 
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X)
tags_covid_data['tags'] = kmeans.labels_
def get_cluster_tags(cluster_num):
    cluster_center = kmeans.cluster_centers_[cluster_num]
    sorted_indices = cluster_center.argsort()[::-1]
    feature_names = vectorizer.get_feature_names_out()
    return [feature_names[i] for i in sorted_indices[:3]]  # Top 3 tags
cluster_tags = {i: get_cluster_tags(i) for i in range(num_clusters)}
print(tags_covid_data[['Q&A', 'tags']])
print("Cluster Tags:")
for cluster, tags in cluster_tags.items():
    print(f"Cluster {cluster}: {tags}")

                                                   Q&A  tags
0    1. Question: When was the Middle East Respirat...     4
1    2. Question: Which medical comorbidities most ...     4
2    3. Question: What is the size of bovine corona...     0
3    4. Question: What is the estimated R0 of COVID...     0
4    5. Question: What is the relationship between ...     4
..                                                 ...   ...
190  191. Question: What suggests that Irish equine...     0
191  192. Question: Where was SARS-CoV-2 first iden...     4
192  193. Question: What is a major difference in c...     4
193  194. Question: How big was the temporary hospi...     0
194  195. Question: Being closest to 2019-nCoV whic...     4

[195 rows x 2 columns]
Cluster Tags:
Cluster 0: ['coronavirus', 'answer', 'question']
Cluster 1: ['ncov', '2019', 'answer']
Cluster 2: ['days', 'person', 'incubation']
Cluster 3: ['case', 'covid', 'jan']
Cluster 4: ['cov', 'sars', 'mers']


In [39]:
print(tags_covid_data.head())

                                                 Q&A  tags
0  1. Question: When was the Middle East Respirat...     4
1  2. Question: Which medical comorbidities most ...     4
2  3. Question: What is the size of bovine corona...     0
3  4. Question: What is the estimated R0 of COVID...     0
4  5. Question: What is the relationship between ...     4
